<a href="https://colab.research.google.com/github/vignesh14analytica/RNN-Word-Embeddings-/blob/master/IMBD_rating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
import io

In [0]:
print(tf.__version__)

**Importing IMDB dataset from Keras datasets**

In [0]:
imbd, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

**Organising data into sentences and labels**

In [0]:
train_data, test_data = imbd['test'], imbd['test']

train_sentences = []
train_labels = []

test_sentences = []
test_labels = []

for s,l in train_data:
  train_sentences.append(s.numpy().decode('utf8'))
  train_labels.append(l.numpy())

for s,l in test_data:
  test_sentences.append(s.numpy().decode('utf8'))
  test_labels.append(l.numpy())

training_labels = np.array(train_labels)
testing_labels = np.array(test_labels)

**Setting up tokenizer, sequencing and Padding data**

In [0]:
vocab_size = 10000
embedding_dim = 16
max_len = 120
trunc_type = 'post'
oov_tok = '<OOV>'

tokenizer = Tokenizer(num_words= vocab_size, oov_token= oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(train_sentences)
padded = pad_sequences(sequences= sequences, maxlen= max_len, truncating= trunc_type)

test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(sequences= test_sequences, maxlen = max_len)

**Reversing word index to decode padded sentences**

In [6]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(train_sentences[3])

poor shirley <OOV> tries hard to lend some <OOV> to this <OOV> gag inducing feel good movie but she's <OOV> by the run away sentimentality of a film that's not the least bit grounded in reality br br this was directed by curtis <OOV> did he have a <OOV> since we last heard from him <OOV> can do effective drama sprinkled with comedy as <OOV> by wonder boys so i don't know what happened to him here this is the kind of movie that doesn't want to accept that life is messy and <OOV> and that neat <OOV> endings however implausible they might be might make for a nice closing shot but come across as utterly phony if the people
Poor Shirley MacLaine tries hard to lend some gravitas to this mawkish, gag-inducing "feel-good" movie, but she's trampled by the run-away sentimentality of a film that's not the least bit grounded in reality.<br /><br />This was directed by Curtis Hanson? Did he have a lobotomy since we last heard from him? Hanson can do effective drama sprinkled with comedy, as evidenc

**Defining model and compiling with Loss function and Optimizer**

In [7]:
model = Sequential([tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_len),
                    tf.keras.layers.Flatten(),
                    tf.keras.layers.Dense(6, activation= 'relu'),
                    tf.keras.layers.Dense(1, activation = 'sigmoid')])

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


**Training the model for 10 epochs**

In [8]:
num_epochs = 10
model.fit(padded, training_labels, epochs = num_epochs, validation_data = (test_padded, testing_labels))

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 0.4817 - accuracy: 0.7461 - val_loss: 0.2828 - val_accuracy: 0.8838
Epoch 2/10
782/782 [==============================] - 3s 4ms/step - loss: 0.2270 - accuracy: 0.9123 - val_loss: 0.2644 - val_accuracy: 0.8881
Epoch 3/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0899 - accuracy: 0.9765 - val_loss: 0.2903 - val_accuracy: 0.8898
Epoch 4/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0229 - accuracy: 0.9970 - val_loss: 0.3365 - val_accuracy: 0.8885
Epoch 5/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0072 - accuracy: 0.9994 - val_loss: 0.3639 - val_accuracy: 0.8906
Epoch 6/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0026 - accuracy: 0.9998 - val_loss: 0.3995 - val_accuracy: 0.8902
Epoch 7/10
782/782 [==============================] - 3s 4ms/step - loss: 9.7221e-04 - accuracy: 1.0000 - val_loss: 0.4272 - val_accuracy: 0.8904
Ep

In [9]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


**Creating the embedding vector and Metadata file**

In [0]:
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

**Download the files**

In [0]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

The word embeddings can be visualized with http://projector.tensorflow.org/